# Линейная алгебра

Подмодуль [scipy.linalg](https://docs.scipy.org/doc/scipy/reference/linalg.html#module-scipy.linalg) в целом является надмножеством над [numpy.linalg](https://numpy.org/doc/stable/reference/routines.linalg.html), т.е. кроме того, что он содержит все методы из `numpy.linalg`, в него включены еще ряд дополнительных методов.

Ряд примеров приведен в [документации](https://docs.scipy.org/doc/scipy/reference/linalg.html#module-scipy.linalg). 
- [Обращение матриц](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html#finding-the-inverse) (функция [linalg.inv](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.linalg.inv.html#scipy.linalg.inv)) и [решение СЛАУ](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html#solving-a-linear-system) (функция [linalg.solve](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.linalg.solve.html));
- [Вычисление норм векторов и матриц](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html#computing-norms);
- [Разложение матрицы](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html#decompositions): собственные значения и вектора, сингулярное разложение, LU и QR разложения и другое;
- [Псевдообращение матриц](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html#generalized-inverse);
- [Методы создания специальных матриц](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html#special-matrices), [блочно-диагональных](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.linalg.block_diag.html#scipy.linalg.block_diag) и многих других;

## Обращение матриц

Функция [linalg.inv](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.linalg.inv.html#scipy.linalg.inv) позволяет обратить матрицу. 

Чтобы убедиться, что она возвращает обратную матрицу, проверим выполнение равенства $A A^{-1} = A^{-1} A = I$, где $I$ --- единичная матрица.

In [1]:
import numpy as np
from scipy import linalg

A = np.array([[1., 2.], [3., 4.]])
A_inv = linalg.inv(A)

print(A, A_inv, sep="\n")

[[1. 2.]
 [3. 4.]]
[[-2.   1. ]
 [ 1.5 -0.5]]


In [2]:
print(A @ A_inv)
print(A_inv @ A)

[[1.0000000e+00 0.0000000e+00]
 [8.8817842e-16 1.0000000e+00]]
[[1.00000000e+00 0.00000000e+00]
 [1.11022302e-16 1.00000000e+00]]


Как видим, получилась не совсем единичная матрица, но т.к. все вычисления с числами с плавающей точкой производятся в компьютере не точно, большего ожидать нельзя. 

Сверять целый массив глазами не очень удобно. Давайте упростим себе задачу и посчитаем норму невязки:

$$
R_1 = || A A^{-1} - I ||
$$

и

$$
R_2 = || A^{-1} A - I ||.
$$


Для этого воспользуемся функцией [scipy.linalg.norm](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.linalg.norm.html#scipy.linalg.norm).

In [3]:
R_1 = linalg.norm(A @ A_inv - np.eye(2))
R_2 = linalg.norm(A_inv @ A - np.eye(2))
print(f"R_1 = {R_1}, R_2 = {R_2}" )

R_1 = 9.930136612989092e-16, R_2 = 2.482534153247273e-16


```{note}
Функция `linalg.norm` позволяет считать разные нормы, в том числе и [матричные](https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D1%80%D0%BC%D0%B0_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D1%8B). 

Если эта функция встречает вектор, то по умолчанию вычисляется евклидова норма, если матрицу, то по умолчанию вычисляется норма Фробениуса.
```

## Решение СЛАУ

Функция [linalg.solve](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.linalg.solve.html) используется для решения СЛАУ вида $Ax = b$, где $A$ --- квадратная матрица размера $N \times N$. Правая часть уравнения $b$ может быть формы `(N, NRHS)`, т.е. если $b$ --- одномерный массив, то решается обычное СЛАУ, а если $b$ --- матрица из $N$ строк и `NRHS` столбцов, то решается матричное уравнение (или сразу решается `NRHS` СЛАУ с общей матрицей $A$, а в качестве векторов правых частей выступают столбцы матрицы $b$). 
 
Для иллюстрации создадим матрицу `A` и вектор `x`. Сгенерируем вектор правой части `b = A @ x` и попробуем восстановить значение `x`, решив СЛАУ `A x = b`.

In [1]:
import numpy as np
from scipy import linalg

A = np.array([[1., 2.], [3., 4.]])
x = np.array([1., 1.])
b = A @ x

x_approx = linalg.solve(A, b)
error = linalg.norm(x_approx - x)
print(f"Погрешность: {error}")

Погрешность: 4.002966042486721e-16


Проиллюстрируем одновременное решение нескольких СЛАУ. Одновременно решим уравнение $A x = b$ и $A y = c$, где $A$ --- общая матрица, а правые части $b$ и $c$ --- разные. Для этого составим матрицу $B$, которая состоит из столбцов $b$ и $c$. Решение $X$ матричного уравнения $A X = B$ --- матрица, столбцы которой --- решения исходных СЛАУ $x$ и $y$.

In [18]:
y = np.array([42., 42.])
c = A @ y
print(f"Точные решения: {x} и {y}")

B = np.vstack([b, c]).T
X = linalg.solve(A, B)

x, y = X.T
print(f"Численные: {x} и {y}")

Точные решения: [1. 1.] и [42. 42.]
Численные: [1. 1.] и [42. 42.]
